In [1]:
# This is necessary for jupyter to draw graphs offline.
# Plotly tries to use cloud service to draw graphs by default.
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
import GPy
import optuna
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.io as pio
import sys
    

seed = 123
np.random.seed(seed=seed)

In [3]:
variance = 1.
lengthscale = 0.2
quantization = 50
epsilon = 0.01
sigma2 = 0.01

In [10]:
kernel = GPy.kern.RBF(input_dim=2, variance=variance, lengthscale=lengthscale)

x = np.linspace(0., 1., quantization)
xx = [[0., 0.] for _ in range(quantization ** 2)]
for i in range(quantization):
    for j in range(quantization):
        xx[i * quantization + j][0] = x[i]
        xx[i * quantization + j][1] = x[j]
xx = np.asarray(xx)
mu_rbf = np.zeros(quantization ** 2)
cov_rbf = kernel.K(xx, xx) + sigma2 * np.eye(quantization ** 2)

# 初期化もサンプリングにしような
# y_rbf = np.zeros(quantization ** 2)
y_rbf = np.random.multivariate_normal(mu_rbf, cov_rbf)
y_rbf_best = []
y_rbf_history = []


def quantize(a):
    return np.argmin(np.abs(np.linspace(0., 1., quantization) - a))


def fun_rbf(a, b):
    global y_rbf
    global y_rbf_best
    global y_rbf_history
    y_rbf = np.sqrt(1. - epsilon) * y_rbf + np.sqrt(epsilon) * np.random.multivariate_normal(mu_rbf, cov_rbf)
    y_rbf_best.append(np.max(y_rbf))
    y_rbf_history.append(y_rbf)
    i = quantize(a)
    j = quantize(b)
    return y_rbf[i + quantization + j]


def objevtive_rbf(trial):
    a, b = trial.suggest_joint('ab',
                               [['uniform', 'a', 0., 1.],
                                ['uniform', 'b', 0., 1.]])
    return fun_rbf(a, b)

In [11]:
n_trials = 300

In [12]:
for i in range(n_trials):
    fun_rbf(0, 0)
    print("\r{}/{} done".format(i+1, n_trials), end='')
print("\n all done!")

300/300 done
 all done!


In [13]:
np.asarray(y_rbf_history).shape

(300, 2500)

In [14]:
np.savetxt('true_bogunovic_rbf.csv', np.asarray(y_rbf_history), delimiter=',')

In [15]:
data = np.loadtxt('true_bogunovic_rbf.csv', delimiter=',')
data.shape

(300, 2500)